# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_file = os.path.join('city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3652764,Puerto Ayora,EC,-0.7393,-90.3518,71.55,93,62,4.00
1,6170031,Tuktoyaktuk,CA,69.4541,-133.0374,51.80,93,75,4.61
2,2264557,Ponta do Sol,PT,32.6667,-17.1000,66.69,76,8,6.46
3,2075265,Busselton,AU,-33.6500,115.3333,61.05,68,91,13.04
4,3899695,Ancud,CL,-41.8697,-73.8203,46.22,89,93,7.54
...,...,...,...,...,...,...,...,...,...
546,6112608,Powell River,CA,49.8496,-124.5361,64.27,84,6,5.41
547,3082704,Wałcz,PL,53.2779,16.4712,64.26,81,8,5.97
548,1489209,Troitskoye,RU,52.9821,84.6757,67.46,52,1,6.80
549,1692364,Puro,PH,12.4841,123.3827,88.00,64,19,9.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
65,3994469,Ojinaga,MX,29.5667,-104.4167,81.55,47,1,6.91
143,3058986,Lučenec,SK,48.3325,19.6671,78.28,56,0,2.53
219,5528182,Pecos,US,31.4229,-103.4932,78.28,57,1,9.22
226,2208791,Yafran,LY,32.0633,12.5286,77.38,34,0,8.68
237,6541176,Agropoli,IT,40.3469,14.9966,80.62,53,4,4.43
276,6540819,Siderno,IT,38.2779,16.3020,82.15,50,0,1.90
287,1506676,Gari,RU,59.4307,62.3497,80.73,39,3,5.17
367,362973,Abnūb,EG,27.2696,31.1511,89.37,31,0,9.51
376,456008,Rūjiena,LV,57.8975,25.3316,76.05,67,5,7.02
386,1082243,Ambilobe,MG,-13.2000,49.0500,78.39,42,1,3.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
65,3994469,Ojinaga,MX,29.5667,-104.4167,81.55,47,1,6.91,
143,3058986,Lučenec,SK,48.3325,19.6671,78.28,56,0,2.53,
219,5528182,Pecos,US,31.4229,-103.4932,78.28,57,1,9.22,
226,2208791,Yafran,LY,32.0633,12.5286,77.38,34,0,8.68,
237,6541176,Agropoli,IT,40.3469,14.9966,80.62,53,4,4.43,
276,6540819,Siderno,IT,38.2779,16.3020,82.15,50,0,1.90,
287,1506676,Gari,RU,59.4307,62.3497,80.73,39,3,5.17,
367,362973,Abnūb,EG,27.2696,31.1511,89.37,31,0,9.51,
376,456008,Rūjiena,LV,57.8975,25.3316,76.05,67,5,7.02,
386,1082243,Ambilobe,MG,-13.2000,49.0500,78.39,42,1,3.76,


In [8]:
# for index, row in hotel_df.iterrows():
#     try:
        
#         base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#         params = {
#         "keyword": "hotel",
#         "radius": 5000,
#         "key": g_key,
#         }


#         lat = row['Lat']
#         lng = row['Lng']

#         params['location'] = f"{lat}, {lng}"

#         hotel_data = requests.get(base_url, params=params).json()
        
#         hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
#     except IndexError:
        
#         hotel_df.loc[index, "Hotel Name"] = "NaN"
        
# hotel_df 

# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
65,3994469,Ojinaga,MX,29.5667,-104.4167,81.55,47,1,6.91,
143,3058986,Lučenec,SK,48.3325,19.6671,78.28,56,0,2.53,
219,5528182,Pecos,US,31.4229,-103.4932,78.28,57,1,9.22,
226,2208791,Yafran,LY,32.0633,12.5286,77.38,34,0,8.68,
237,6541176,Agropoli,IT,40.3469,14.9966,80.62,53,4,4.43,
276,6540819,Siderno,IT,38.2779,16.3020,82.15,50,0,1.90,
287,1506676,Gari,RU,59.4307,62.3497,80.73,39,3,5.17,
367,362973,Abnūb,EG,27.2696,31.1511,89.37,31,0,9.51,
376,456008,Rūjiena,LV,57.8975,25.3316,76.05,67,5,7.02,
386,1082243,Ambilobe,MG,-13.2000,49.0500,78.39,42,1,3.76,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))